In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [13]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[-1], True)
    print("GPU available")
else:
    print("GPU not available, using CPU")

GPU available


In [14]:
def AlexNetScratch(input_shape=(224, 224, 3), num_classes=10):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (11, 11), strides=4, padding='same', activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((3, 3), strides=2))
    model.add(layers.Conv2D(192, (5, 5), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D((3, 3), strides=2))
    model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D((3, 3), strides=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [15]:
def resize_images(images, new_size):
    resized_images = tf.image.resize(images, new_size)
    return resized_images

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertir los datos a tensores
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)

# Normalizar los datos
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convertir las etiquetas a one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [16]:
# Create tf.data.Dataset objects
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Resize images on-the-fly during training
train_dataset = train_dataset.map(lambda image, label: (resize_images(image, (224, 224)), label))
test_dataset = test_dataset.map(lambda image, label: (resize_images(image, (224, 224)), label))

# Batch the datasets
batch_size = 64  # Adjust as needed
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [18]:
learning_rate = 0.001
num_epochs = 10

model = AlexNetScratch()
model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_dataset, epochs=num_epochs, batch_size=batch_size, validation_data=test_dataset)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


782/782 ━━━━━━━━━━━━━━━━━━━━ 62s 73ms/step - accuracy: 0.2192 - loss: 2.0836 - val_accuracy: 0.4638 - val_loss: 1.4619
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 63ms/step - accuracy: 0.4683 - loss: 1.4612 - val_accuracy: 0.5270 - val_loss: 1.3144
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 63ms/step - accuracy: 0.5403 - loss: 1.2864 - val_accuracy: 0.5623 - val_loss: 1.2134
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 80s 61ms/step - accuracy: 0.5834 - loss: 1.1749 - val_accuracy: 0.6030 - val_loss: 1.1265
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 47s 60ms/step - accuracy: 0.6122 - loss: 1.0916 - val_accuracy: 0.6107 - val_loss: 1.1110
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 64ms/step - accuracy: 0.6397 - loss: 1.0181 - val_accuracy: 0.6282 - val_loss: 1.0725
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 80s 62ms/step - accuracy: 0.6634 - loss: 0.9530 - val_accuracy: 0.6262 - val_loss: 1.1164
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 84s 64ms/step - accuracy: 0.6770 - loss: 0.9148 - val_accurac

In [19]:
model_path = "alexnet_scratch_tensorflow.h5"
model.save(model_path)